<a href="https://colab.research.google.com/github/hezarai/notebooks/blob/main/hezar/01_models_overview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Models
In Hezar, models are the typical PyTorch modules with some extra features for loading, saving, exporting, etc.
Let's dive into some of the most important ones!



## Models Basics
### Building Models
Like any other package, you can import any model from `hezar.models` that you want.


In [1]:
from hezar.models import BertLM, BertLMConfig

bert = BertLM(BertLMConfig())

/home/aryan/Applications/miniconda3/envs/main/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


You can also configure the architecture by changing the properties in a model's config like so:

In [2]:
config = BertLMConfig(num_hidden_layers=8, num_attention_heads=8)
bert = BertLM(config)


Every model in Hezar, can be pushed to or downloaded from the Hub.

### Loading pre-trained models
Loading a model from Hub is as easy as:

In [3]:
from hezar.models import Model

bert = Model.load("hezarai/bert-base-fa")

The `load` methods takes the following steps to build the model:

1. Load the config file `model_config.yaml` and figure out the model's class using the `name` config parameter. (`bert_lm` in this snippet)
2. Build the model with random weights from the corresponding class. (`BertLM` in this snippet)
3. Download the weights file (`model.pt`) and load the state dict into to the model.
4. If the path contains any preprocessor, the preprocessor (`WordPieceTokenizer` in this snippet) will be loaded too.
(You can disable loading preprocessors by setting `Model.load(path, load_preprocessor=False)`)


### Inference & Prediction
Now that you have loaded a model along with its preprocessors, feature extractors, etc. you can perform an end-to-end
inference in a single line of code using `Model.predict` method.

A sequence labeling example would be like this:

In [4]:
from hezar.models import Model

pos_model = Model.load("hezarai/bert-fa-pos-lscp-500k")  # Part-of-speech
inputs = ["شرکت هوش مصنوعی هزار"]
pos_outputs = pos_model.predict(inputs)
print(f"POS: {pos_outputs}")

POS: [[{'token': 'شرکت', 'label': 'Ne'}, {'token': 'هوش', 'label': 'Ne'}, {'token': 'مصنوعی', 'label': 'AJe'}, {'token': 'هزار', 'label': 'NUM'}]]



### Saving Models
You can save any model along with its config and preprocessor and other files on disk like:

In [5]:
from hezar.models import RobertaLM, RobertaLMConfig

roberta = RobertaLM(RobertaLMConfig(vocab_size=60000))
roberta.save("my-roberta")


### Pushing to the Hub
Every model can be pushed to the Hub.

In [ ]:
from hezar.models import RobertaTextClassification, RobertaTextClassificationConfig

roberta = RobertaTextClassification(RobertaTextClassificationConfig(num_labels=2))
roberta.push_to_hub("arxyzan/roberta-sentiment")

model.pt:   1%|          | 4.55M/473M [00:19<09:47, 798kB/s]


## Custom Models
Every Hezar model is a subclass of the base model class `Model` and the `Model` itself is a subclass of PyTorch `nn.Module`
with some extra features. So if you're familiar with PyTorch, this should feel like home!


### A Sample Perceptron

In [ ]:
from dataclasses import dataclass

import torch
from torch import Tensor, nn

from hezar.models import Model, ModelConfig, register_model


@dataclass
class PerceptronConfig(ModelConfig):
    name = "perceptron"
    input_shape: int = 4
    output_shape: int = 2


@register_model("perceptron", config_class=PerceptronConfig)
class Perceptron(Model):
    """
    A simple single layer network
    """

    def __init__(self, config, **kwargs):
        super().__init__(config, **kwargs)
        self.nn = nn.Linear(
            in_features=self.config.input_shape,
            out_features=self.config.output_shape,
        )

    def preprocess(self, raw_inputs):
        inputs_tensor = Tensor(raw_inputs)
        return inputs_tensor

    def forward(self, x: torch.Tensor):
        x = x.reshape(1, -1)
        x = self.nn(x)
        return x

    def post_process(self, model_outputs: torch.Tensor):
        return model_outputs.cpu().numpy()


The only point here is that you have to pass a `ModelConfig` to your model and read everything from the config and the
rest is just typical PyTorch stuff.

In [12]:
model = Perceptron(PerceptronConfig())
inputs = [1, 2, 3, 4]
outputs = model.predict(inputs)
print(outputs)

[[-1.5652378  0.7015635]]


Now you have a Hezar compatible model!

In [13]:
model.save("my-perceptron")
model.push_to_hub("arxyzan/perceptron")

Hezar (INFO): Uploaded: `PerceptronConfig()` --> `arxyzan/perceptron/model_config.yaml`
model.pt: 100%|██████████| 1.55k/1.55k [00:00<00:00, 4.15kB/s]
Hezar (INFO): Uploaded: `Perceptron(name=perceptron)` --> `arxyzan/perceptron/model.pt`


To learn more about the internals of the models in Hezar take a look at [the models in-depth guide](../guide/models_advanced.md)